### Train a bilstm to generate jokes in forward and reverse based on a controlled bit

In [28]:
import pickle
import importlib
from library import data_preprocess as dp
importlib.reload(dp)
import random
from time import time

import numpy as np

import keras
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Input
from keras.layers import Dense, Dropout, Activation
from keras.layers import LSTM, Bidirectional
from keras.layers import Embedding, TimeDistributed, Flatten, Merge, Concatenate
from keras import regularizers
from keras.metrics import sparse_categorical_accuracy, sparse_categorical_crossentropy
from keras.models import load_model
from keras.optimizers import Adam
from keras.models import Model

from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
import tensorflow as tf
from keras import backend as K
from keras.utils import multi_gpu_model

### Variables

In [2]:
# http://nlp.stanford.edu/data/glove.6B.zip
DATA_PATH = './datasets/combined.pickle'
VOCAB_PATH = './datasets/combined_vocabulary.pickle'
MODELS_PATH = './models/'

GLOVE_PATH = './data/glove.6B.200d.txt'

MODEL_PREFIX = 'combined_controlled_stacked_lstm_glove'

MAX_SEQUENCE_LENGTH = 13
VALIDATION_SPLIT = 0.2

GLOVE_EMBEDDING_DIM = 200
EMBEDDING_DIM1 = 512
EMBEDDING_DIM2 = 512
HIDDEN_DIM1 = 1024
HIDDEN_DIM2 = 512
DEEPER_DIM = 512
DROPOUT_FACTOR = 0.2
REGULARIZATION = 0.00001
LEARNING_RATE = 0.003

DATA_PERCENT = 0.1

RUN_INDEX = 1

In [3]:
with open(DATA_PATH, 'rb') as pickleFile:
    sentences = pickle.load(pickleFile)

with open(VOCAB_PATH, 'rb') as pickleFile:
    vocab = pickle.load(pickleFile)
    
random.shuffle(sentences)

print("Number of sentences = ", len(sentences))
print(sentences[:2])
print("Vocab size = ", len(vocab))
print(vocab[:10])

Number of sentences =  270543
[[2, 'sos i think that was a long enough break back to work ! eos'], [2, 'sos i heard that george was in nyc this weekend and i missed him ! omg ! i could just die ! eos']]
Vocab size =  12614
['liberate', 'savings', 'clip', 'rhino', 'cross', 'encouragement', 'viewed', 'womens', 'lap', 'challenges']


In [4]:
NUM_JOKES = len([0 for item in sentences if item[0] == 0])
NUM_QUOTES = len([1 for item in sentences if item[0] == 1])
NUM_TWEETS = len([2 for item in sentences if item[0] == 2])
print("Number of Jokes: ", NUM_JOKES)
print("Number of Quotes: ", NUM_QUOTES)
print("Number of Tweets: ", NUM_TWEETS)
TOTAL_SENTENCES = NUM_JOKES + NUM_QUOTES + NUM_TWEETS
JOKES_WEIGHT = TOTAL_SENTENCES/NUM_JOKES
QUOTES_WEIGHT = TOTAL_SENTENCES/NUM_QUOTES
TWEETS_WEIGHT = TOTAL_SENTENCES/NUM_TWEETS
print("JOKES_WEIGHT: ", JOKES_WEIGHT)
print("QUOTES_WEIGHT: ", QUOTES_WEIGHT)
print("TWEETS_WEIGHT: ", TWEETS_WEIGHT)

CLASS_WEIGHTS = [JOKES_WEIGHT, QUOTES_WEIGHT, TWEETS_WEIGHT]
CLASSES = [item[0] for item in sentences]
sentences = [item[1] for item in sentences]

Number of Jokes:  96910
Number of Quotes:  43383
Number of Tweets:  130250
JOKES_WEIGHT:  2.7916933237024044
QUOTES_WEIGHT:  6.236152409930157
TWEETS_WEIGHT:  2.07710556621881


In [5]:
# tokenize data
num_words = len(vocab)

tokenizer = Tokenizer(num_words=None, filters='', lower=True, split=' ', 
                      char_level=False, oov_token=None)
tokenizer.fit_on_texts(sentences)
assert num_words == len(tokenizer.word_index)

encoded_sentences = tokenizer.texts_to_sequences(sentences)
print(encoded_sentences[:5])

VOCAB_SIZE = len(tokenizer.word_index) + 1
print(VOCAB_SIZE)

[[1, 5, 78, 20, 34, 7, 158, 304, 502, 80, 6, 59, 16, 2], [1, 5, 416, 20, 3420, 34, 17, 2341, 37, 206, 9, 5, 633, 137, 16, 499, 16, 5, 164, 32, 442, 16, 2], [1, 51, 11, 899, 641, 35, 7, 5102, 13, 282, 340, 1230, 24, 4, 324, 8, 8325, 24, 4, 577, 8, 9, 146, 21, 120, 3075, 43, 17, 262, 4337, 3, 2], [1, 626, 20, 2327, 10, 97, 3, 183, 3, 534, 3, 427, 3, 129, 3, 2], [1, 28, 10, 497, 1263, 22, 1841, 12, 9, 4, 45, 34, 2328, 2]]
12615


In [6]:
# saving
with open(MODELS_PATH + MODEL_PREFIX + '_tokenizer_' + str(RUN_INDEX) + '.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Preparing Training Data

In [20]:
TAG_SIZE = 3
X_data = []
y_data = []
tag_data = []
weight_data = []

for idx, sentence in enumerate(encoded_sentences):
    l = len(sentence)
    sliding_window_length = min(l-3, MAX_SEQUENCE_LENGTH)
    step_size = 1
    for i in range(0, l - sliding_window_length, step_size):
        X_data.append(sentence[i:i+sliding_window_length])
        y_data.append(sentence[i+1:i+sliding_window_length+1])
        tag_data.append(CLASSES[idx])
        weight_data.append(CLASS_WEIGHTS[CLASSES[idx]])
        
print("Total training data size = ", len(X_data))
MAX_SEQ_LEN = max([len(seq) for seq in X_data])
print("Max seq len = ", MAX_SEQ_LEN)

X_data = pad_sequences(X_data, maxlen=MAX_SEQ_LEN, padding='pre')
y_data = pad_sequences(y_data, maxlen=MAX_SEQ_LEN, padding='pre').reshape(-1, MAX_SEQ_LEN, 1)
tag_data = to_categorical([[[tag]]*MAX_SEQ_LEN for tag in tag_data], TAG_SIZE)
weight_data = np.array(weight_data).reshape(-1,)
#y_data = np.array(y_data).reshape(-1,1)

# shuffle
perm = np.random.permutation(X_data.shape[0])
X_data = X_data[perm]
y_data = y_data[perm]
tag_data = tag_data[perm]
weight_data = weight_data[perm]
print(X_data.shape)
print(X_data[:5])
print(y_data.shape)
print(y_data[:5])
print(tag_data.shape)
print(tag_data[:5])
print(weight_data.shape)
print(weight_data[:5])

Total training data size =  2244907
Max seq len =  13
(2244907, 13)
[[  94   61   19  221    3  353   21    6  194   63  324  249   65]
 [ 691   23   22   58   23  390    8  223 1342   38   68 3464   13]
 [  14   69   26   20   70  498   13   18  498  579  433 2364   19]
 [   8   76   10  959    9   31   80   43   16 1787   16  812   91]
 [   5  146   30   17  139  284   16  499   10  386   11    7  439]]
(2244907, 13, 1)
[[[  61]
  [  19]
  [ 221]
  [   3]
  [ 353]
  [  21]
  [   6]
  [ 194]
  [  63]
  [ 324]
  [ 249]
  [  65]
  [   3]]

 [[  23]
  [  22]
  [  58]
  [  23]
  [ 390]
  [   8]
  [ 223]
  [1342]
  [  38]
  [  68]
  [3464]
  [  13]
  [  23]]

 [[  69]
  [  26]
  [  20]
  [  70]
  [ 498]
  [  13]
  [  18]
  [ 498]
  [ 579]
  [ 433]
  [2364]
  [  19]
  [ 704]]

 [[  76]
  [  10]
  [ 959]
  [   9]
  [  31]
  [  80]
  [  43]
  [  16]
  [1787]
  [  16]
  [ 812]
  [  91]
  [   2]]

 [[ 146]
  [  30]
  [  17]
  [ 139]
  [ 284]
  [  16]
  [ 499]
  [  10]
  [ 386]
  [  11]
  [   7]

In [8]:
print('Indexing glove word vectors')
#Glove Vectors
glove_embeddings_index = {}
f = open(GLOVE_PATH)
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    glove_embeddings_index[word] = coefs
f.close()
print('Total %s word vectors.' % len(glove_embeddings_index))

Indexing glove word vectors
Total 400000 word vectors.


In [9]:
print('Preparing glove embedding matrix')
glove_embedding_matrix = np.zeros((VOCAB_SIZE, GLOVE_EMBEDDING_DIM))
for word,i in tokenizer.word_index.items():
    embedding_vector = glove_embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        glove_embedding_matrix[i] = embedding_vector
print('Null word embeddings: %d' % np.sum(np.sum(glove_embedding_matrix, axis=1) == 0))
print(glove_embedding_matrix.shape)

Preparing glove embedding matrix
Null word embeddings: 298
(12615, 200)


In [15]:
from keras.layers import merge
from keras.layers.core import Lambda
from keras.models import Model

import tensorflow as tf

def make_parallel(model, gpu_count):
    def get_slice(data, idx, parts):
        shape = tf.shape(data)
        size = tf.concat([ shape[:1] // parts, shape[1:] ],axis=0)
        stride = tf.concat([ shape[:1] // parts, shape[1:]*0 ],axis=0)
        start = stride * idx
        return tf.slice(data, start, size)

    outputs_all = []
    for i in range(len(model.outputs)):
        outputs_all.append([])

    #Place a copy of the model on each GPU, each getting a slice of the batch
    for i in range(gpu_count):
        with tf.device('/gpu:%d' % i):
            with tf.name_scope('tower_%d' % i) as scope:

                inputs = []
                #Slice each input into a piece for processing on this GPU
                for x in model.inputs:
                    input_shape = tuple(x.get_shape().as_list())[1:]
                    slice_n = Lambda(get_slice, output_shape=input_shape, arguments={'idx':i,'parts':gpu_count})(x)
                    inputs.append(slice_n)                

                outputs = model(inputs)
                
                if not isinstance(outputs, list):
                    outputs = [outputs]
                
                #Save all the outputs for merging back together later
                for l in range(len(outputs)):
                    outputs_all[l].append(outputs[l])

    # merge outputs on CPU
    with tf.device('/cpu:0'):
        merged = []
        for outputs in outputs_all:
            merged.append(merge(outputs, mode='concat', concat_axis=0))
            
        return Model(input=model.inputs, output=merged)

In [40]:
# define model
def StackedLSTM(vocab_size, glove_embedding_dim, glove_embedding_matrix, embedding_dim1, embedding_dim2,
           hidden_dim1, hidden_dim2, deeper_dim, max_seq_len, tag_size,
           dropout_factor=0.5, regularization=0.00001, learning_rate=0.001):
    
    inputs = Input(shape=(None,))
    tag_inputs = Input(shape=(None,tag_size))
    
    glove_embedding = Embedding(vocab_size, glove_embedding_dim, #input_length=max_seq_len,
                                  weights=[glove_embedding_matrix],
                                  mask_zero=True,trainable=False)(inputs)
    
    word_embedding = Embedding(vocab_size, embedding_dim1, #input_length=max_seq_len, 
                               mask_zero=True, embeddings_regularizer=regularizers.l2(regularization))(inputs)
    
    #tag_embedding = Embedding(tag_size, tag_size, embeddings_regularizer=regularizers.l2(regularization))(tag_inputs)
    
    concat_embeds = Concatenate(axis=-1)([glove_embedding, word_embedding, tag_inputs])
    
    final_embed = Dense(units=embedding_dim2, activation='tanh',
                        kernel_regularizer=regularizers.l2(regularization))(concat_embeds)
    
    lstm1 = LSTM(hidden_dim1, activation='tanh', 
                   kernel_regularizer=regularizers.l2(regularization), 
                   recurrent_regularizer=regularizers.l2(regularization), #unroll=True, 
                   return_sequences = True, dropout=dropout_factor, recurrent_dropout=dropout_factor)(final_embed)
    
    lstm2 = LSTM(hidden_dim2, activation='tanh', 
                   kernel_regularizer=regularizers.l2(regularization), 
                   recurrent_regularizer=regularizers.l2(regularization), #unroll=True, 
                   return_sequences = True, dropout=dropout_factor, recurrent_dropout=dropout_factor)(lstm1)
    
    timedist_dropout = TimeDistributed(Dropout(dropout_factor))(lstm2)
    
    deep_dense = Dense(units=deeper_dim, activation='tanh', 
                       kernel_regularizer=regularizers.l2(regularization))(timedist_dropout)
    
    dropout_layer1 = Dropout(dropout_factor)(deep_dense)
    
    outputs = Dense(units=vocab_size, activation='softmax', 
                    kernel_regularizer=regularizers.l2(regularization))(dropout_layer1)
    
    model = Model(inputs=[inputs,tag_inputs], outputs=outputs)
    #model = make_parallel(Model(inputs=[inputs,tag_inputs], outputs=outputs), 2)
    #model = multi_gpu_model(model, gpus=2)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=learning_rate),
                  metrics=[sparse_categorical_crossentropy, sparse_categorical_accuracy]#, sample_weight_mode='temporal'
                 )
    return model

In [41]:
K.clear_session()
config = tf.ConfigProto(allow_soft_placement=True)
sess = tf.Session(config=config)
K.set_session(sess)

model = StackedLSTM(vocab_size=VOCAB_SIZE, glove_embedding_dim=GLOVE_EMBEDDING_DIM,
                    glove_embedding_matrix=glove_embedding_matrix, 
                    embedding_dim1=EMBEDDING_DIM1, embedding_dim2=EMBEDDING_DIM2,
                    hidden_dim1=HIDDEN_DIM1, hidden_dim2=HIDDEN_DIM2,
                    deeper_dim=DEEPER_DIM, max_seq_len=MAX_SEQ_LEN, dropout_factor=DROPOUT_FACTOR, 
                    regularization=REGULARIZATION, learning_rate=LEARNING_RATE, tag_size=TAG_SIZE)
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 200)    2523000     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 512)    6458880     input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 3)      0                                            
__________________________________________________________________________________________________
concatenat

In [42]:
class TB(TensorBoard):
    def __init__(self, log_every=1, **kwargs):
        super().__init__(**kwargs)
        self.log_every = log_every
        self.counter = 0
    
    def on_batch_end(self, batch, logs=None):
        self.counter+=1
        if self.counter%self.log_every==0:
            for name, value in logs.items():
                if name in ['batch', 'size']:
                    continue
                summary = tf.Summary()
                summary_value = summary.value.add()
                summary_value.simple_value = value.item()
                summary_value.tag = name
                self.writer.add_summary(summary, self.counter)
            self.writer.flush()
        
        super().on_batch_end(batch, logs)

In [ ]:
start_time = time()
tensorboard = TB(log_dir="./logs/" + MODEL_PREFIX + "/{}".format(time()), 
                          histogram_freq=0, write_graph=True, write_images=False, log_every=10)

callbacks=[tensorboard, 
           EarlyStopping(patience=5, monitor='val_loss'),
           ModelCheckpoint(filepath=MODELS_PATH + 'checkpoints/'+ MODEL_PREFIX + '_gen' + str(RUN_INDEX) + '.{epoch:02d}-{val_loss:.2f}.hdf5', 
                           monitor='val_loss', verbose=1, mode='auto', period=1), 
           ModelCheckpoint(filepath=MODELS_PATH + MODEL_PREFIX + '_gen'+str(RUN_INDEX)+'.hdf5', 
                           monitor='val_loss', verbose=1, mode='auto', period=1, save_best_only=True)]

model.fit([X_data, tag_data], y_data, epochs=10, batch_size=1024, shuffle=True, 
          verbose=1, validation_split=0.2, callbacks=callbacks, sample_weight=weight_data)

print("Total elapsed time: ", time()-start_time)

Train on 1795925 samples, validate on 448982 samples
Epoch 1/10
1795925/1795925 [==============================] - 2751s 2ms/step - loss: 18.5659 - sparse_categorical_crossentropy: 5.3421 - sparse_categorical_accuracy: 0.1383 - val_loss: 15.3502 - val_sparse_categorical_crossentropy: 4.2757 - val_sparse_categorical_accuracy: 0.2304

Epoch 00001: saving model to ./models/checkpoints/combined_controlled_stacked_lstm_glove_gen1.01-15.35.hdf5

Epoch 00001: val_loss improved from inf to 15.35020, saving model to ./models/combined_controlled_stacked_lstm_glove_gen1.hdf5
Epoch 2/10
1795925/1795925 [==============================] - 2749s 2ms/step - loss: 15.5209 - sparse_categorical_crossentropy: 4.2997 - sparse_categorical_accuracy: 0.2262 - val_loss: 14.6837 - val_sparse_categorical_crossentropy: 4.0368 - val_sparse_categorical_accuracy: 0.2536

Epoch 00002: saving model to ./models/checkpoints/combined_controlled_stacked_lstm_glove_gen1.02-14.68.hdf5

Epoch 00002: val_loss improved from 15

In [38]:
#model.save_weights(MODELS_PATH + 'checkpoints/'+ MODEL_PREFIX + '_gen' + str(RUN_INDEX)+'_epoch1.hdf5')

In [39]:
# generate a sequence from a language model
def generate_categorical(model, tokenizer, seed_text, maxlen, probabilistic=False, exploration_factor=1.0, tag=0):
    
    reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))
    seq = tokenizer.texts_to_sequences([seed_text])[0]
    print(seq)
    
    while True:
        encoded_seq = seq
        if len(seq) > MAX_SEQ_LEN:
            encoded_seq = encoded_seq[-1*MAX_SEQ_LEN:]
            
        #padded_seq = pad_sequences([encoded_seq], maxlen=MAX_SEQ_LEN, padding='pre')
        padded_seq = np.array([seq])
        tags = to_categorical(np.full((1, padded_seq[0].shape[0]), tag), TAG_SIZE)
        y_prob = model.predict([padded_seq,tags])[0][-1].reshape(1,-1)#[3:].reshape(-1,1)
        
        if random.random() <= exploration_factor:
            probabilistic = True
        else:
            probabilistic = False
            
        if probabilistic:
            y_class = np.argmax(np.random.multinomial(1,y_prob[0]/(np.sum(y_prob[0])+1e-5),1))
        else:
            y_class = y_prob.argmax(axis=-1)[0]
        
        if y_class == 0:
            break
        out_word = reverse_word_map[y_class]
        seq.append(y_class)
        if out_word == 'eos' or len(seq) > maxlen or out_word == 'sos':
            break
    
    words = [reverse_word_map[idx] for idx in seq]
    
    return ' '.join(words)

In [69]:
joke = generate(model, tokenizer, "sos i had to use", maxlen=40)
print(joke)

[1, 7, 84, 8, 258]


ValueError: Error when checking : expected embedding_12_input to have shape (10,) but got array with shape (5,)

In [55]:
def bigrams_list(sentence):
    words = sentence.split(' ')
    bigrams = []
    for i in range(0, len(words)-1):
        bigrams.append(words[i]+' '+words[i+1])
    return bigrams

print(bigrams_list("sos hello , i'm a dinosaur . eos"))

['sos hello', 'hello ,', ", i'm", "i'm a", 'a dinosaur', 'dinosaur .', '. eos']


In [56]:
sentence_bigrams = [bigrams_list(s) for s in sentences]
print(sentence_bigrams[:2])

[['sos i', 'i had', 'had to', 'to use', 'use my', 'my glasses', 'glasses when', 'when playing', 'playing tennis', 'tennis .', '. because', 'because its', 'its a', 'a no', 'no contact', 'contact sport', 'sport .', '. eos'], ['sos why', 'why did', 'did the', 'the japanese', 'japanese funeral', 'funeral home', 'home have', 'have to', 'to turn', 'turn away', 'away new', 'new business', 'business ?', '? they', 'they ran', 'ran out', 'out of', 'of san', 'san storage', 'storage eos']]


In [57]:
def intersection(lst1, lst2):
    temp = set(lst2)
    lst3 = [value for value in lst1 if value in temp]
    return lst3

def similarity_score(lst1, lst2):
    intersection_len = len(intersection(lst1, lst2))
    return (1.0*intersection_len)/len(lst1)#+len(lst2)-intersection_len)
 
def print_closest_sentences(sentence, sentence_bigrams, top_k=3):
    bigrams = bigrams_list(sentence)
    scores = np.array([similarity_score(bigrams, sbigrams)
                       for sbigrams in sentence_bigrams])
    top_k_indices = scores.argsort()[-1*top_k:][::-1]
    top_k_scores = scores[top_k_indices]
    for k in range(top_k):
        print(top_k_scores[k], " -> ", sentences[top_k_indices[k]])

In [62]:
print_closest_sentences(joke, sentence_bigrams, 10)

0.6363636363636364  ->  sos i made up a new word . plagiarism . eos
0.6363636363636364  ->  sos i just invented a new word . plagiarism . eos
0.6363636363636364  ->  sos i invented a new word . plagiarism . eos
0.6363636363636364  ->  sos i just invented a new joke . i just invented a new word . plagiarism . eos
0.6363636363636364  ->  sos i made a new joke . i made a new word . plagiarism . eos
0.5454545454545454  ->  sos i recently invented a new word to describe a lot of the jokes on the subreddit . plagiarism . eos
0.5454545454545454  ->  sos i invented a new word the other day . plagiarism . eos
0.5454545454545454  ->  sos hey people , i've invented a new word . plagiarism . eos
0.5454545454545454  ->  sos i created a new word today . plagiarism . eos
0.45454545454545453  ->  sos i had to use my glasses when playing tennis . because its a no contact sport . eos


In [65]:
joke = generate(model, tokenizer, "sos what do you call", maxlen=40)
print(joke)
print_closest_sentences(joke, sentence_bigrams, 10)

[1, 17, 20, 9, 66]
sos what do you call a vegetarian ? a rip - off . eos
0.8461538461538461  ->  sos what do you call a cheap circumcision ? a rip - off . eos
0.8461538461538461  ->  sos what do you call a cheap circumcision ? a rip - off . well , you can't blame them . they don't make much money , they just keep the tips . eos
0.8461538461538461  ->  sos what do you call a bad hairdresser who is also very expensive ? a rip - off . eos
0.6923076923076923  ->  sos what do you call a cheap circumcision ? rip - off . eos
0.6923076923076923  ->  sos what do you call a cheap circumcision ? a rip - off ! eos
0.6923076923076923  ->  sos what do you call a gay vegetarian ? a vegetarian . eos
0.6923076923076923  ->  sos what do you call a bad circumcision ? a rip - off eos
0.6923076923076923  ->  sos what do you call a bad circumcision ? what do you call a bad circumcision ? a rip off . eos
0.6923076923076923  ->  sos what do you call a discount circumcision ? a rip off . eos
0.6923076923076923

In [64]:
print(sentences[:10])

['sos i had to use my glasses when playing tennis . because its a no contact sport . eos', 'sos why did the japanese funeral home have to turn away new business ? they ran out of san storage eos', 'sos a world without women would be a pain in the ass . eos', 'sos not everyone that comes into your life needs to stay there . eos', 'sos " i would absolutely say i\'m an introvert ! " - guy screaming to his table full of friends at brunch . eos', 'sos cashier : " would you like to donate to charity today or are you a giant piece of shit ? " eos', "sos what do you call a muslim girl dating an agnostic guy ? for safety purposes , i don't know if i should tell you her name threedots eos", "sos math is so communist threedots threedots there's class struggle for marx eos", "sos i used to be a bodybuilder threedots or ' the dr frankenstein grave robber ' as the press preferred to call me . eos", "sos i can't believe this paper went to college , let alone thought it ruled eos"]
